In [1]:
# from pyspark.sql.functions import udf, col
# from pyspark.sql.types import StringType, DoubleType



from time import sleep

import pyspark
from IPython.display import clear_output, display
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.streaming import StreamingQuery
from pyspark.sql.types import DateType, IntegerType, StringType, StructType
from pyspark.sql.functions import min, max, avg
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType


In [2]:
spark = (SparkSession.builder.appName("Assignment_5.3").getOrCreate())

# spark = SparkSession.builder \
#     .appName("Assignment_5.3") \
#     .master("local[*]") \
#     .getOrCreate()

23/05/18 10:01:58 WARN Utils: Your hostname, all-MS-7D35 resolves to a loopback address: 127.0.1.1; using 192.168.1.108 instead (on interface enp2s0)
23/05/18 10:01:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/18 10:01:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/18 10:02:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Read data from source to DataFrame
df = spark.read.format("csv").option("header","True").option("inferschema", "true").load("Documents/Assignment_5.3/titanic.csv")

In [4]:
df.printSchema()

root
 |-- 10: integer (nullable = true)
 |-- 01: integer (nullable = true)
 |-- 3: integer (nullable = true)
 |-- Braund, Mr. Owen Harris: string (nullable = true)
 |-- male: string (nullable = true)
 |-- 22: integer (nullable = true)
 |-- 16: integer (nullable = true)
 |-- 07: integer (nullable = true)
 |-- A/5 21171: string (nullable = true)
 |-- 7.25: double (nullable = true)
 |-- _c10: string (nullable = true)
 |-- S: string (nullable = true)
 |-- 2020-01-01 13:45:25: timestamp (nullable = true)



In [5]:
df.show()

+---+---+---+-----------------------+------+----+---+---+----------------+-------+----+---+-------------------+
| 10| 01|  3|Braund, Mr. Owen Harris|  male|  22| 16| 07|       A/5 21171|   7.25|_c10|  S|2020-01-01 13:45:25|
+---+---+---+-----------------------+------+----+---+---+----------------+-------+----+---+-------------------+
|  2|  1|  1|   Cumings, Mrs. Joh...|female|  38|  1|  0|        PC 17599|71.2833| C85|  C|2020-01-01 13:44:48|
|  3|  1|  3|   Heikkinen, Miss. ...|female|  26|  0|  0|STON/O2. 3101282|  7.925|null|  S|2020-01-01 13:38:11|
|  4|  1|  1|   Futrelle, Mrs. Ja...|female|  35|  1|  0|          113803|   53.1|C123|  S|2020-01-01 13:32:00|
|  5|  0|  3|   Allen, Mr. Willia...|  male|  35|  0|  0|          373450|   8.05|null|  S|2020-01-01 13:36:30|
|  6|  0|  3|       Moran, Mr. James|  male|null|  0|  0|          330877| 8.4583|null|  Q|2020-01-01 13:31:39|
|  7|  0|  1|   McCarthy, Mr. Tim...|  male|  54|  0|  0|           17463|51.8625| E46|  S|2020-01-01 13

23/05/18 10:02:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 1, 0, 3, Braund, Mr. Owen Harris, male, 22, 1, 0, A/5 21171, 7.25, , S, 2020-01-01 13:45:25
 Schema: 10, 01, 3, Braund, Mr. Owen Harris, male, 22, 16, 07, A/5 21171, 7.25, _c10, S, 2020-01-01 13:45:25
Expected: 10 but found: 1
CSV file: file:///home/syedmuhammadraqimali/Documents/Assignment_5.3/titanic.csv


In [6]:
old_column_names = df.columns

col_names=["PassengerId","Survived","Pclass","Name","Sex","Age","SibSp","Parch","Ticket","Fare","Cabin","Embarked","Date"]


In [7]:
for i in range(len(old_column_names)):
    df = df.withColumnRenamed(old_column_names[i], col_names[i])

In [8]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Date: timestamp (nullable = true)



In [9]:
# Calculate statistics for numerical columns


numerical_cols=[]

for col_name, data_type in df.dtypes:
    if data_type in ["int", "bigint", "double", "float"]:
        numerical_cols.append(col_name)

In [10]:
numerical_cols

['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [11]:

numerical_stats = df.select(*numerical_cols).describe().toPandas()
numerical_stats = numerical_stats.set_index("summary")
numerical_stats = numerical_stats.astype(float)


23/05/18 10:02:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/05/18 10:02:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 1, 0, 3, 22, 1, 0, 7.25
 Schema: 10, 01, 3, 22, 16, 07, 7.25
Expected: 10 but found: 1
CSV file: file:///home/syedmuhammadraqimali/Documents/Assignment_5.3/titanic.csv


In [12]:
numerical_stats

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
summary,,,,,,,
count,890.000000,890.000000,890.000000,713.000000,890.000000,890.000000,890.000000
mean,446.500000,0.384270,2.307865,29.690042,0.522472,0.382022,32.232246
stddev,257.065167,0.486696,0.836220,14.543836,1.103247,0.806409,49.714317
min,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [13]:

# Calculate min, max, and avg for numerical columns using Spark functions
df.select([min(col).alias("min_" + col) for col in numerical_cols] + \
          [max(col).alias("max_" + col) for col in numerical_cols] + \
          [avg(col).alias("avg_" + col) for col in numerical_cols]).show()

23/05/18 10:02:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 1, 0, 3, 22, 1, 0, 7.25
 Schema: 10, 01, 3, 22, 16, 07, 7.25
Expected: 10 but found: 1
CSV file: file:///home/syedmuhammadraqimali/Documents/Assignment_5.3/titanic.csv


+---------------+------------+----------+-------+---------+---------+--------+---------------+------------+----------+-------+---------+---------+--------+---------------+------------------+------------------+------------------+------------------+-------------------+-----------------+
|min_PassengerId|min_Survived|min_Pclass|min_Age|min_SibSp|min_Parch|min_Fare|max_PassengerId|max_Survived|max_Pclass|max_Age|max_SibSp|max_Parch|max_Fare|avg_PassengerId|      avg_Survived|        avg_Pclass|           avg_Age|         avg_SibSp|          avg_Parch|         avg_Fare|
+---------------+------------+----------+-------+---------+---------+--------+---------------+------------+----------+-------+---------+---------+--------+---------------+------------------+------------------+------------------+------------------+-------------------+-----------------+
|              2|           0|         1|      0|        0|        0|     0.0|            891|           1|         3|     80|        8|      

In [14]:
str_columns = ["PassengerId","Sex","Cabin","Embarked"]
def change_last_letter_after_space(word):
    if word is not None:
        words = word.split()
        for i in range(len(words)):
            words[i] = words[i][:-1] + "1"
        return " ".join(words)
    return word
change_last_letter_udf = udf(change_last_letter_after_space, StringType())
# Apply the UDF to each categorical column
for column in str_columns:
    df = df.withColumn(column, change_last_letter_udf(df[column]))
# Show the modified DataFrame
df.show()









23/05/18 10:02:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 1, 0, 3, Braund, Mr. Owen Harris, male, 22, 1, 0, A/5 21171, 7.25, , S, 2020-01-01 13:45:25
 Schema: 10, 01, 3, Braund, Mr. Owen Harris, male, 22, 16, 07, A/5 21171, 7.25, _c10, S, 2020-01-01 13:45:25
Expected: 10 but found: 1
CSV file: file:///home/syedmuhammadraqimali/Documents/Assignment_5.3/titanic.csv
23/05/18 10:02:08 ERROR Executor: Exception in task 0.0 in stage 9.0 (TID 7)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_121997/1649325756.py", line 4, in change_last_letter_after_space
AttributeError: 'int' object has no attribute 'split'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:561)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:94)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_121997/1649325756.py", line 4, in change_last_letter_after_space
AttributeError: 'int' object has no attribute 'split'
